In [24]:
import os
import numpy as np
from collections import Counter
from sklearn import svm
from sklearn.metrics import accuracy_score
import pickle
import gzip
TRAIN_DIR="C:\\Users\\super\\Desktop\\Eckovation\\SVM_Dataset\\train-mails"
TEST_DIR="C:\\Users\\super\\Desktop\\Eckovation\\SVM_Dataset\\test-mails"


In [25]:
#load function for loading the model
def load(file_name):
    stream=gzip.open(file_name,"rb")
    model=pickle.load(stream)
    stream.close()
    return model
    

In [26]:
#saving the model
def save(file_name,model):
    stream=gzip.open(file_name,"wb")
    pickle.dump(model,stream)
    stream.close()
    

In [27]:
def make_dict(root_dir):
    all_words=[]
    emails=[os.path.join(root_dir,f)for f in os.listdir(root_dir)]
    for mail in emails:
        with open(mail) as m:
            for line in m:
                words=line.split()
                all_words+= words
    dictionary=Counter(all_words)
    list_to_remove=list(dictionary)
    
    for item in list_to_remove:
        if item.isalpha()==False:
            del dictionary[item]
        elif len(item)==1:
            del dictionary[item]
    dictionary=dictionary.most_common(3000)
    return dictionary
        
            

In [28]:
def extract_feature(mail_dir):
    files=[os.path.join(mail_dir,fi)for fi in os.listdir(mail_dir)]
    features_matrix=np.zeros((len(files),3000))
    train_labels=np.zeros(len(files))
    count=0;
    docID=0;
    
    for fil in files:
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i==2:
                    words=line.split()
                    for word in words:
                        wordID=0
                        for i,d in enumerate(dictionary):
                            if d[0]==word:
                                wordID=i
                                features_matrix[docID,wordID]=words.count(word)
        train_labels[docID]=0;
        filepathtokens=fil.split('/')
        lasttoken=filepathtokens[len(filepathtokens)-1]
        if lasttoken.__contains__("spmsg"):
            train_labels[docID]=1;
            count=count+1
        docID=docID+1
    return features_matrix,train_labels
        

In [29]:
dictionary=make_dict(TRAIN_DIR)
print("Reading and Processing emails from file.")


Reading and Processing emails from file.


In [30]:
features_matrix,labels=extract_feature(TRAIN_DIR)
test_feature_matrix,test_labels=extract_feature(TEST_DIR)


In [31]:
model=svm.SVC(kernel="rbf",C=100,gamma=0.001)
print("Training model.")


Training model.


In [32]:
model.fit(features_matrix,labels)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [34]:
predicted_labels=model.predict(test_feature_matrix)
print("Finished classifying,Accuracy Score:")
print(accuracy_score(test_labels,predicted_labels))

Finished classifying,Accuracy Score:
0.9730769230769231
